In [92]:
import os
from os import listdir
from pathlib import *
from pathlib import Path
import cv2 as cv
import numpy as np
from PIL import Image
import matplotlib

matplotlib.use('TKAgg')
import shutil
from matplotlib import pyplot as plt
from pixelmatch.contrib.PIL import pixelmatch as pm2

In [24]:
matplotlib.get_backend()

'TkAgg'

# Image lines segmentation

Find the edges of the image (i.e. the edges of the single lines).


In [56]:
def get_line(img, dr):
    img = cv.imread(img)
    grey = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    contours, hierarchy = cv.findContours(grey, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    line_num = len(contours)
    print("Number of line elements found: " + str(line_num))

    for x in range(len(contours)):
        #selecting line x of those detected
        bb = contours[x]
        #creating a stencil of the original image
        #1. preparing the black background image;
        stencil = np.zeros(img.shape).astype(img.dtype)
        #2. set into an array the actual contours of the 'line element';
        cont = [np.array(bb)]
        #3. fill the contours with white color (i.e. [255, 255, 255]);
        line = cv.fillPoly(stencil, cont, color=(255, 255, 255))

        #_._._Saving step
        dr = './' + str(dr) + '/'
        filename = dr + 'line' + str(x) + '.png'
        cv.imwrite(filename, line)

    return (line_num)

In [47]:
#dumb fun to count the num of .png file on a directory
def elements(dr):
    count = 0
    for x in os.listdir(dr):
        if x.endswith(".png"):
            count += 1
    return (count)

# Intersection over Union

In [82]:
#Defining a function to calculate IoU on the fly
def IoU(l1, l2):
    intersection = np.logical_and(l1, l2)
    union = np.logical_or(l1, l2)
    iou_score = np.sum(intersection) / np.sum(union)
    return (iou_score)

# Recall and Percision

Procedure: https://www.jeremyjordan.me/evaluating-image-segmentation-models/

In [109]:
def recall(gt, prd):

    #setting variables
    score = []
    TP = 0
    FN = 0
    threshold = .45
    dir1 = Path(gt).glob('*.png')
    dir2 = Path(prd).glob('*.png')

    #number of lines detected on the gt image
    gtn = sum(1 for _ in dir1)
    #number of lines detected on the pred image
    pn = sum(1 for _ in dir2)
    xtr_ln = abs(pn - gtn)
    print('gt lines: ' + str(gtn))
    print('prd lines: ' + str(pn))

    #Iterating through the gt lines found
    for idx, img in enumerate(os.listdir(gt)):
        if (img.endswith(".png")):
            image1 = cv.imread(gt + img)

            print(idx, idx, img)

            #Iterating through the prd lines found
            for idx2, img2 in enumerate(os.listdir(prd)):
                if (img2.endswith(".png")):
                    image2 = Image.open(prd + img2)

                    #print(idx2, img2)

                    score.append(IoU(image1, image2))

            #Calculating FP and FN through predefined threshold
            if max(score) > threshold:
                TP += 1
            else:
                FN += 1
            score = []

    Rec = TP / (TP + FN)

    #out = "Recall score: " + str(Rec)

    return (Rec)

In [110]:
def precision(gt, prd):

    #setting variables
    score = []
    TP = 0
    FP = 0
    threshold = .45
    dir1 = Path(gt).glob('*.png')
    dir2 = Path(prd).glob('*.png')

    #number of lines detected on the gt image
    gtn = sum(1 for _ in dir1)
    #number of lines detected on the pred image
    pn = sum(1 for _ in dir2)
    print('gt lines: ' + str(gtn))
    print('prd lines: ' + str(pn))

    #Iterating through the gt lines found
    for idx, img in enumerate(os.listdir(prd)):
        if (img.endswith(".png")):
            image1 = cv.imread(prd + img)

            print(idx, idx, img)

            #Iterating through the prd lines found
            for idx2, img2 in enumerate(os.listdir(gt)):
                if (img2.endswith(".png")):
                    image2 = Image.open(gt + img2)

                    #print(idx2, img2)

                    score.append(IoU(image1, image2))

            #Calculating FP and FN through predefined threshold
            if max(score) > threshold:
                TP += 1
            else:
                FP += 1
            score = []

    Precision = TP / (TP + FP)

    #out = "Precision score: " + str(Precision)

    return (Precision)

In [101]:
def get_fname(dir):
    filename = dir.split('/')
    filename = filename[-1]
    return(filename)

# Main

In [118]:
import os
import glob

def main():

    n=0
    rec=0
    prec=0
    direc= os.getcwd()

    for img in glob.glob(direc + '/gt/*.png'):
        store_r=0
        store_p=0
        for img2 in glob.glob(direc + '/output/*.png'):
            if get_fname(img) == get_fname(img2):
              #_._.create working folders
              os.mkdir(direc + '/linegt')
              os.mkdir(direc + '/lineprd')
              get_line(img, 'linegt')
              get_line(img2, 'lineprd')
              lin_gt = direc + '/linegt/'
              lin_prd = direc + '/lineprd/'
              store_r = recall(lin_gt, lin_prd)
              rec = rec + store_r
              print('Recall: ' + str(store_r))
              store_p = precision(lin_gt, lin_prd)
              prec = prec + store_p
              print('Precision: ' + str(store_p))
              n+=1
              #_._.remove working folders
              shutil.rmtree(direc + '/linegt')
              shutil.rmtree(direc + '/lineprd')

    rec_mean = rec/n
    prec_mean = prec/n
    reca='\n\nRecall mean over '+ str(n) + ' documents: '+ str(rec_mean)
    preci= '\n\nPrecision mean over '+ str(n) + ' documents: '+ str(prec_mean)
    out = reca + preci
    print(out)
    return ()

In [117]:
if __name__ == "__main__":
    main()

Number of line elements found: 72
Number of line elements found: 81
gt lines: 72
prd lines: 81
0 0 line40.png
1 1 line54.png
2 2 line68.png
3 3 line1.png
4 4 line0.png
5 5 line69.png
6 6 line55.png
7 7 line41.png
8 8 line57.png
9 9 line43.png
10 10 line2.png
11 11 line3.png
12 12 line42.png
13 13 line56.png
14 14 line52.png
15 15 line46.png
16 16 line7.png
17 17 line6.png
18 18 line47.png
19 19 line53.png
20 20 line45.png
21 21 line51.png
22 22 line4.png
23 23 line5.png
24 24 line50.png
25 25 line44.png
26 26 line23.png
27 27 line37.png
28 28 line36.png
29 29 line22.png
30 30 line34.png
31 31 line20.png
32 32 line21.png
33 33 line35.png
34 34 line19.png
35 35 line31.png
36 36 line25.png
37 37 line24.png
38 38 line30.png
39 39 line18.png
40 40 line26.png
41 41 line32.png
42 42 line33.png
43 43 line27.png
44 44 line16.png
45 45 line17.png
46 46 line15.png
47 47 line29.png
48 48 line28.png
49 49 line14.png
50 50 line38.png
51 51 line10.png
52 52 line11.png
53 53 line39.png
54 54 line13.pn